# Python私房手册-有点坑的logging

Python自带的logging确实很强大，看起来很容易使用，但是坑很多，非常容易出错。 

## 基础使用

python的logging日志记录模块非常强大，使用也很简单，但是特别容易出各种意外状况，打印各种出乎意料的log。最近对logging的一些原理进行了学习，再此做个记录，以备忘。  
首先全面的了解一下整体的结构。logging默认就有一个root的Logger对象，输入logging.root可以看到，默认为warning级别：  
```python
>>> logging.root
<RootLogger root (WARNING)>
```
用户自行创建的所有logger对象，都是root的子对象。
```
>>> logger = logging.getLogger('logger')
>>> logger.parent
<RootLogger root (WARNING)>
```
需要注意的是，当getLogger()不带参数时，返回的就是rootLogger对象本身。
```Python
>>> logger = logging.getLogger()
>>> logger
<RootLogger root (WARNING)>
>>> logger is logging.root
True
```
创建了Logger对象，接下来我们需要创建handler对象，handler对象是用来配置处理log时用的格式，级别等等特性的，我们可以根据需求创建各种不同的handler，比如将log记录保存在文件的FileHandler，将log输出到屏幕的StreamHandler，支持日志根据时间回滚的TimedRotatingFileHandler，根据文件大小回滚的RotatingFileHandler等等（回滚不太好理解，根据时间或文件大小回滚其实就是根据时间或者文件大小来保存日志，比如只保存3天的日志，超过3天的就自动删除，或者设置日志文件只能为一定大小，超过就删除）。  
各种handler网上各种文章都有介绍，或者查官方文档，这里就不列举了。稍微一提的是，最常用的FileHandler和StreamHandler在logging下，其它的handler在logging.handlers下。另外，logger和handler的level可以分别设置，以满足不同的需求。    
创建了handler，就要把它添加到logger对象中去，logger对象有一个handlers属性，其实就是一个简单的列表，可以看到所有添加给它的hanlder。如下：
```Python
>>> logger = logging.getLogger('spam')
>>> handler = logging.StreamHandler()
>>> logger.addHandler(handler)
>>> logger.handlers
[<StreamHandler <stderr> (NOTSET)>]
```
可见，logger对象添加了handler，是一个输出到stderr，级别未设置的handler。此时，就可以通过logger对象输出各种logger了，如：
```Python
>>> logger = logging.getLogger('spam')
>>> handler = logging.StreamHandler()
>>> formatter = logging.Formatter("%(asctime)s-%(levelname)s-%(message)s")
>>> handler.setFormatter(formatter)
>>> handler.setLevel(logging.INFO)
>>> logger.addHandler(handler)
>>> logger.handlers
[<StreamHandler <stderr> (INFO)>]
>>> logger.setLevel(logging.INFO)
>>> logger.info('info')
2019-03-31 11:59:41,933-INFO-info
```

## logger

### 打印参数

使用logging打印消息的时候，除了输入`message`，有几个地方要注意：
1. 可以输入多个位置参数，除第一个是message外，后面的都对应着message的格式化参数，如：

In [7]:
import logging

logging.basicConfig(level=logging.DEBUG)
logging.info("%s wrong happend!", "TypeError")

INFO:root:TypeError wrong happend!


2. 如果要打印异常，可以传入一个`exc_info`的关键字参数，该参数可以等于某个异常实例或者异常三元组，也可以是布尔值，如果为True，默认取`sys.exc_info`返回的三元组，另外，要捕获异常的话，必须放在`try`这样包含异常的上下文中。

In [8]:
import logging

try:
    raise KeyError("key error")
except Exception:
    logging.error("found error", exc_info=True)

ERROR:root:found error
Traceback (most recent call last):
  File "<ipython-input-8-4ecf9644cac4>", line 4, in <module>
    raise KeyError("key error")
KeyError: 'key error'


3. 除了打印异常，还可以打印堆栈，只需要设置`stack_info`，注意堆栈不是异常，不需要在`try`这样的上下文中，另外3.8以后还有个`stacklevel`关键字参数，可以指定打印的层级。

## filter

### 类方法

除了定制不同级别的level，logger还可以添加filter对象来对输出进行更加复杂的控制。filter和handler一样，先进行配置，然后再添加到logger对象中。所有日志在输出之前，都会先通过filter进行过滤。创建filter要先创建一个继承自logging.Filter的类，代码如下：

In [4]:
import logging
import sys


class ContextFilter(logging.Filter):
    def filter(self, record):
        if record.role == "admin":
            return True
        else:
            return False


logger = logging.getLogger("Wechat")
logger.setLevel(logging.DEBUG)
handler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter(
                '%(asctime)s %(levelname)s:%(message)s Role:%(role)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

f = ContextFilter()
logger.addFilter(f)
logger.info('An info message with %s', 'some parameters',
            extra={"role": "admin"})
logger.info('An info message with %s', 'some parameters',
            extra={"role": "hacker"})

2021-07-08 11:14:30,883 INFO:An info message with some parameters Role:admin


INFO:Wechat:An info message with some parameters


### 函数方法

从3.2以后，不一定非要创建一个logging.Filter子类，只要是有filter属性的任何对象或者函数都行，后台会检查这个对象是否有filter属性，如果有，则会调用这个对象的filter()方法，没有的话就把它当作一个callable对象，直接执行，并且把records当作一个参数传递给这个callable对象。
```Python
import logging
import sys


def myfilter(record):
    if record.role == "admin":
        return True
    else:
        return False


if __name__ == '__main__':
    logger = logging.getLogger("Wechat")
    logger.setLevel(logging.DEBUG)
    handler = logging.StreamHandler(sys.stdout)
    formatter = logging.Formatter(
            '%(asctime)s %(levelname)s: %(message)s Role: %(role)s')
    handler.setFormatter(formatter)
    logger.addHandler(handler)
    logger.addFilter(myfilter)
    logger.info('An info message with %s', 'some parameters',
                extra={"role":"admin"})
    logger.info('An info message with %s', 'some parameters',
                extra={"role":"hacker"}
```
需要注意的是，如果使用上面的例子，那么在每一条日志记录里面，都需要加上一条extra={"role":"xxx"}，因为所有的日志record对象都会先通过filter过滤，此时会检查record是否有role属性，如果没有设置，显然会报错。  
其实除了过滤这一种用法，filter还可以通过在函数里面给record添加属性，方便的增加自定义的字段。还是看例子：
```Python
def myfilter(record):
	record.user = 'telecomshy'
	return True

>>> logger = logging.getLogger()
>>> logger.addFilter(myfilter)
>>> import sys
>>> fmt = logging.Formatter("%(levelname)s-%(asctime)s-%(message)s-%(user)s")
>>> handler = logging.StreamHandler(sys.stdout)
>>> handler.setLevel("DEBUG")
>>> handler.setFormatter(fmt)
>>> logger.addHandler(handler)
>>> logger.setLevel("DEBUG")
>>> logger.info('hello world')
INFO-2019-04-05 11:55:28,764-hello world-telecomshy
```
如上，通过使用filter，现在可以在formatter里面使用'user'这个自定义的字段了。

## 常见问题

### 为什么log会输出重复内容

首先，平时在log输出的时候，经常会发现，莫名奇妙的会输出重复的内容，主要有以下几个可能的原因：  
1. 前面说过，任何自定义的logger对象都是rootLogger的子对象，就像这样，rootLogger(parent)->Logger(child)，而当你使用自定义的logger记录日志的时候，它会从子到父，从左到右依次执行所有的handler来输出，看代码：
```Python
>>> rootLogger = logging.getLogger()
>>> rootLogger.setLevel(logging.INFO)
>>> roothandler = logging.StreamHandler()
>>> fmt = logging.Formatter("%(name)s-%(levelname)s-%(message)s")
>>> roothandler.setLevel(logging.INFO)
>>> roothandler.setFormatter(fmt)
>>> rootLogger.addHandler(roothandler)
>>> rootLogger.handlers
[<StreamHandler <stderr> (INFO)>]
>>> childLogger = logging.getLogger("child")
>>> childLogger.setLevel(logging.INFO)
>>> childhandler = logging.StreamHandler()
>>> childhandler.setLevel(logging.INFO)
>>> childhandler.setFormatter(fmt)
>>> childLogger.addHandler(childhandler)
>>> childLogger.handlers
[<StreamHandler <stderr> (INFO)>]
>>> childLogger.info('i am child')
child-INFO-i am child
child-INFO-i am child
```
日志输出了2次，注意输出的logger name，虽然有一个是rootLogger的handler输出的，但是name还是子logger的。

2. 有人可能会说，上面这个情况不太可能会发生，因为我直接设置一个子logger就ok了，是这样没错，but，当你直接使用logging.basicConfig对rootLogger对象进行配置或者在创建自己的logger对象之前，用logging.info等命令输出过日志的时候，logging会自动的（偷偷摸摸的）给你创建一个streamhandler，如下：
```Python
>>> root = logging.getLogger()
>>> root.handlers
[]
>>> logging.basicConfig(level=logging.INFO)
>>> root.handlers
[<StreamHandler <stderr> (NOTSET)>]
```
```Python
>>> root = logging.getLogger()
>>> root.handlers
[]
>>> logging.info('i am root')
>>> root.handlers
[<StreamHandler <stderr> (NOTSET)>]
```
现在明白为什么有时候创建了一个logger以后，会莫名其妙重复多次输出日志了吧？  
再来看一个官网的例子：
```python
FORMAT = '%(asctime)-15s %(clientip)s %(user)-8s %(message)s'
logging.basicConfig(format=FORMAT)
d = {'clientip': '192.168.0.1', 'user': 'fbloggs'}
logger = logging.getLogger('tcpserver')
logger.warning('Protocol problem: %s', 'connection reset', extra=d)
```
输出为：
```python
2021-07-07 23:32:45,858 192.168.0.1 fbloggs  Protocol problem: connection reset
```
只有一行。查看logger.handlers为空，查看logger.root.handlers为`[<StreamHandler <stderr> (NOTSET)>]`。所以，实际上logger是没有handler的，warning消息向上传递，由root的handler处理了。真的是很容易出错啊。

### 为什么某些IDE的log记录会越来越多

3、还有一种情况，也堪称巨坑。当使用IDE工具编程的时候，整个IDE会话不结束，logger的handlers列表不清空！这就导致反复运行程序的时候，handler会反复添加，结果你会发现log输出越来越多。。。。。如下：
![%E5%9B%BE%E7%89%87.png](attachment:%E5%9B%BE%E7%89%87.png)
第一次运行，一个handler，第二次运行，变2个了，怎么办呢？可以在程序结尾加一行代码root.removeHandler(handler)，或者简单粗暴一点，直接root.handlers=[]，每次程序运行完，将handlers列表全清空。  
差不多就这些吧，最后还有一个小知识点，有些同学会问，为啥我自定义的logger啥handler都没加的时候，也有输出呢？比如：
```Python
>>> import logging
>>> logger = logging.getLogger('child')
>>> logger.handlers
[]
>>> logging.root.handlers
[]
>>> logger.warning('i am child')
i am child
```
因为logging模块有一个默认的hanlder，可以通过logging.lastResort查看，如下：
```Python
>>> logging.lastResort
<_StderrHandler <stderr> (WARNING)>
```
这个handler没有和任何logger关联，专门处理用户啥都没配置的情况，可见，默认级别是warning，默认输出是stderr（注意是stderr而不是标准的stdout，因此如果你采取这种方式输出日志又进行了重定向，很有可能达不到想要的效果），仅仅输出一个message，其它啥都没有。   

### 为什么basicConfig不起作用

某些情况下baseConfig设置不起作用，如下：

```python
import logging

logging.info("hello world!")  # 不会打印
logging.basicConfig(level=logging.INFO)
logging.info("hello world!")  # 仍然不打印，basicConfig设置不起作用
```
但是这样又可以打印了，不能运行上一行代码，否则不打印：

In [1]:
import logging
logging.basicConfig(level=logging.INFO)
logging.info("hello world!")  # basicConfig设置起作用了
logging.basicConfig(level=logging.WARNING)
logging.info("hello world!")  # basicConfig设置又不起作用了，仍然打印

INFO:root:hello world!
INFO:root:hello world!


个人推测，使用`basicConfig`进行配置的时候，会首先检查rootLogger是否包含handler，如果已经包含handler，则不会对rootLogger的level进行修改，如果不包含，则会给rootLogger添加一个没有设置级别的`StreamHandler`，然后再修改logger的level。

而如果你先运行了类似`logging.info`这样的打印语句，会偷偷给`rootLogger`添加一个没有设置级别的`StreamHandler`，因此再使用`basicConfig`设置`level`就不起作用了。